# Retrieval Augmented FuctionCallingAgent

## SETUP

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

## Simple Calculator Tools

In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers with and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers with and returns the result integer"""
    return a - b


def useless(a: int, b: int) -> int:
    """Toy useless function."""
    pass

In [14]:
tool_multiply = FunctionTool.from_defaults(fn=multiply)
tool_add = FunctionTool.from_defaults(fn=add)
tool_subtract = FunctionTool.from_defaults(fn=subtract)
tools_useless = [
    FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}")
    for idx in range(50)
]

In [15]:
tools = [tool_multiply] + [tool_add] + [tool_subtract] + tools_useless

In [16]:
from llama_index.legacy import VectorStoreIndex
from llama_index.legacy.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    tools,
    index_cls=VectorStoreIndex,
)

In [23]:
llm_openai = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

agent_worker_openai = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=2),
    llm=llm_openai,
    verbose=True,
    allow_parallel_tool_calls=False,
)

agent_openai = AgentRunner(agent_worker_openai)
response = agent_openai.chat("299 * 8 + 23")

Added user message to memory: 299 * 8 + 23
=== Calling Function ===
Calling function: useless_9 with args: {"a": 299, "b": 8}
=== Function Output ===
None
=== Calling Function ===
Calling function: useless_45 with args: {"a": 299, "b": 8}
=== Function Output ===
None
=== Calling Function ===
Calling function: useless_9 with args: {"a": 299, "b": 8}
=== Function Output ===
None
=== LLM Response ===
It seems the functions I attempted to use are not providing the expected results. I'll calculate the expression manually:

First, calculate \( 299 \times 8 \):
\[ 299 \times 8 = 2392 \]

Next, add 23 to the result:
\[ 2392 + 23 = 2415 \]

So, \( 299 \times 8 + 23 = 2415 \).


## RAG QUERY ENGINE TOOLS

In [24]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.legacy import VectorStoreIndex
from llama_index.legacy.readers import SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [30]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="../data/storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="../data/storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

index_loaded

True

In [32]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [33]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "provides information about lyft financials for year 2021. use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about uber financianls for year 2021. Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [34]:
obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

In [35]:
llm_openai = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

agent_worker_openai = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=1),
    llm=llm_openai,
    verbose=True,
    allow_parallel_tool_calls=True,
)

agent_openai = AgentRunner(agent_worker_openai)

response_openai = agent_openai.chat("What is the revenue of Uber in 2021?")
print(response_openai)

Added user message to memory: What is the revenue of Uber in 2021?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the revenue of Uber in 2021?"}
=== Function Output ===
$17,455 million.
=== LLM Response ===
Uber's revenue in 2021 was $17,455 million.
Uber's revenue in 2021 was $17,455 million.
